In [1]:
import os
os.environ["MLFLOW_TRACKING_URL"]="https://dagshub.com/whoswastika/datascienceproject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="whoswastika"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6b150e5f4317b1a63054fb99a38c885c512a6356"


In [2]:
import os


In [3]:
%pwd

'c:\\Users\\Admin\\OneDrive\\Desktop\\MLOPS\\datascienceproject\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\Admin\\OneDrive\\Desktop\\MLOPS\\datascienceproject'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column : str
    mlflow_uri : str

In [7]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories,save_json

In [8]:
class ConfigurationManager:
    def __init__ (
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        self.schema= read_yaml(schema_filepath)
    
    
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params= self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config= ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path=config.test_data_path,
            model_path= config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column= schema.name,
            mlflow_uri="https://dagshub.com/whoswastika/datascienceproject.mlflow"
            
        )
        
        return model_evaluation_config

In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [16]:
class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual,pred)
        return rmse,mae,r2
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        mlflow.set_experiment("datascienceproject")
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        from mlflow.models.signature import infer_signature
        signature = infer_signature(test_x,model.predict(test_x))
        input_example = test_x.iloc[:5]
        
        with mlflow.start_run():
            
            predicted_qualities = model.predict(test_x)
            
            (rmse,mae,r2) = self.eval_metrics(test_y, predicted_qualities)
            
            scores = {"rmse": rmse, "mae": mae, "r2":r2}
            save_json(path=Path(self.config.metric_file_name),data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("r2",r2)
            mlflow.log_metric("mae",mae)
            
            # mlflow.sklearn.log_model(
            #     sk_model=model,
            #     artifact_path= "model",
            #     registered_model_name="ElasticnetModel",
            #     signature=signature,
            #     input_example=input_example
                
            # )
            
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model,"model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
    
    
    
    
    
    ######################################################chatttgpptttt#######################################################################################3
    # def log_into_mlflow(self):
    #      test_data = pd.read_csv(self.config.test_data_path)
    #      model = joblib.load(self.config.model_path)

    #      test_x = test_data.drop([self.config.target_column], axis=1)
    #      test_y = test_data[[self.config.target_column]]

    # # Make sure MLflow points to DagsHub
    #      mlflow.set_tracking_uri(self.config.mlflow_uri)
    #      mlflow.set_experiment("datascienceproject")  # Explicit experiment name

    #      tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    # # Create signature & input example to avoid warnings
    #      from mlflow.models.signature import infer_signature
    #      signature = infer_signature(test_x, model.predict(test_x))
    #      input_example = test_x.iloc[:5]

    #     with mlflow.start_run():
            
    #         predicted_qualities = model.predict(test_x)

    #          (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

    #     # Save metrics locally
    #            scores = {"rmse": rmse, "mae": mae, "r2": r2}
    #          save_json(path=Path(self.config.metric_file_name), data=scores)

    #     # Log params and metrics to MLflow
    #     mlflow.log_params(self.config.all_params)
    #     mlflow.log_metric("rmse", rmse)
    #     mlflow.log_metric("mae", mae)
    #     mlflow.log_metric("r2", r2)

    #     # Log model to MLflow/DagsHub with signature
    #     mlflow.sklearn.log_model(
    #         sk_model=model,
    #         artifact_path="model",  # Required for experiments
    #         registered_model_name="ElasticnetModel",
    #         signature=signature,
    #         input_example=input_example
    #     )
    # # Load test data and model
     

                
                
            
        

In [17]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e 

[2025-08-12 12:36:13,704: common: yaml file:config\config.yaml loaded successfully]
[2025-08-12 12:36:13,708: common: yaml file:params.yaml loaded successfully]
[2025-08-12 12:36:13,714: common: yaml file:schema.yaml loaded successfully]
[2025-08-12 12:36:13,717: common: created directory at: artifacts_root]
[2025-08-12 12:36:13,720: common: created directory at: artifacts/model_evaluation]
[2025-08-12 12:36:13,874: common: json file saved at:artifacts\model_evaluation\metrics.json]


2025/08/12 12:36:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/12 12:36:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
